In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import warnings
warnings.filterwarnings(
    "ignore",
    message=".*",
    category=UserWarning
)

In [2]:
import superheader as sup
import os
import torch
import torch.nn as nn
import torch.optim as optim
import TRAIN.architecture.BERT.bert as bert
import live

I0000 00:00:1751285466.699244  912775 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751285466.705081  912990 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751285466.709274  912990 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1751285466.712553  912775 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M3 Pro
W0000 00:00:1751285466.774284  913001 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751285466.780970  913003 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for fe

In [3]:
TRAIN_classes = 'all-classes'

for data_unit in [sup.DATA_S_PF, sup.DATA_S_PV]:
    for PH2 in [False, True]:
        for PH3 in [False, True]:

            # --- Phase-dependent config ---
            if PH3:
                reducer_name = sup.PH3_REDUCER_NAME_PCA
                n_components = 15
            else:
                reducer_name = ''
                n_components = 75 if PH2 else 72

            # --- Build model ---
            data_config = {
                "data_unit" : data_unit,
                "label_col" : sup.class_numeric_column,
                "class_list" : TRAIN_classes,
                "batch_size" : 1024,
                "PH2" : PH2,
                "PH3" : PH3,
                "kernel" : '',
                "reducer" : reducer_name,
                "n" : n_components,
            }

            train_config = {
                "arch" : sup.TRAIN_BERT_CODE,
                "device" : bert.device,
                "loadable" : bert.BERT_MINI,
                "optimizer" : optim.AdamW,
                "lr" : 1e-5,
                "weight_decay" : 0,
                "loss_fn" : nn.CrossEntropyLoss,
                "num_epochs" : 1
            }

            model = bert.BERT(data_config=data_config, df=None, train_config=train_config)

            model_path_file = live.bert_model_paths[TRAIN_classes][data_unit][PH2][PH3]
            full_model_path = os.path.join(live.model_path_root, sup.TRAIN_BERT_CODE, data_unit, live.bertmini_path, model_path_file)
            model.me.load_state_dict(torch.load(full_model_path, map_location='cpu'))
            model.me.eval()

            model.test()
            model.full_score()

            print(f"\n\n{data_unit}; {PH2}; {PH3}")
            print(model.accuracy)
            print(model.top2accuracy)
            print(model.macro_f1)
            print(model.macro_precision)
            print(model.macro_recall)



Spf; False; False
0.9925237637509345
0.9983979493752002
0.992519320839558
0.99254575537255
0.9925313676288786


Spf; False; True
0.9906013030011749
0.9964754886254406
0.9906055922355024
0.9906208040268175
0.9906460594221521


Spf; True; False
0.9850475275018691
0.9942326177507209
0.9850580454334469
0.9850689790220789
0.9851046339325451


Spf; True; True
0.9762896507529638
0.9903876962512015
0.9763674819230912
0.9764651964397035
0.9763601281404072


Spv; False; False
0.9756722151088348
0.9833546734955185
0.975609256545901
0.9771544554153249
0.9753423279739069


Spv; False; True
0.9500640204865557
0.9782330345710627
0.9478736594927473
0.9558114204330569
0.9496548496548496


Spv; True; False
0.9308578745198464
0.9564660691421255
0.9309490183675817
0.9337879674553522
0.9308509808509807


Spv; True; True
0.9167733674775929
0.93854033290653
0.9173687469084615
0.920320634485394
0.9172848172848173


In [4]:
# --- Build model ---
data_config = {
    "data_unit" : sup.DATA_S_PF,
    "label_col" : sup.class_numeric_column,
    "class_list" : TRAIN_classes,
    "batch_size" : 1024,
    "PH2" : False,
    "PH3" : False,
    "kernel" : '',
    "reducer" : '',
    "n" : 72,
}

train_config = {
    "arch" : sup.TRAIN_BERT_CODE,
    "device" : bert.device,
    "loadable" : bert.BERT_MINI,
    "optimizer" : optim.AdamW,
    "lr" : 1e-5,
    "weight_decay" : 0,
    "loss_fn" : nn.CrossEntropyLoss,
    "num_epochs" : 1
}

model = bert.BERT(data_config=data_config, df=None, train_config=train_config)

model_path_file = live.bert_model_paths[TRAIN_classes][sup.DATA_S_PF][False][False]
full_model_path = os.path.join(live.model_path_root, sup.TRAIN_BERT_CODE, sup.DATA_S_PF, live.bertmini_path, model_path_file)
model.me.load_state_dict(torch.load(full_model_path, map_location='cpu'))
model.me.eval()

model.test()
model.full_score()

print(f"\n\n{data_unit}; {PH2}; {PH3}")
print(model.accuracy)
print(model.top2accuracy)
print(model.macro_f1)
print(model.macro_precision)
print(model.macro_recall)

model.full_df



Spv; True; True
0.9942326177507209
0.9982911460002136
0.9942262791922629
0.9942027156384985
0.9942661499720794


,fileid,person_id,cycle_num,handedness,class_name,class_numeric,current_frame,h0x,h0y,h0z,...,p0x,p0y,p0z,p11x,p11y,p11z,p12x,p12y,p12z,active_hand
0,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,0,0.413438,0.516854,-9.053193e-08,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,1
1,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,0,0.568475,0.911143,1.136299e-09,...,0.498476,0.318532,-0.423539,0.577536,0.519796,-0.141686,0.425831,0.492205,-0.158985,0
2,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,1,0.411601,0.525609,-9.427173e-08,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,1
3,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,1,0.568473,0.911162,1.245373e-09,...,0.497645,0.315444,-0.436276,0.577870,0.520437,-0.134161,0.424087,0.495226,-0.161746,0
4,p01/Ciclo_1_5_Derecha/Ciclo_1_5_Derecha_1.mp4,p01,1,0,1,28,2,0.409486,0.530432,-8.504437e-08,...,0.497244,0.318666,-0.457492,0.579072,0.523931,-0.151225,0.423043,0.499277,-0.169967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91746,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,9,0.632517,0.482991,-1.874786e-07,...,0.484490,0.289181,-0.421546,0.572918,0.474172,-0.078406,0.397896,0.484275,-0.113990,1
91747,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,10,0.634008,0.486989,-1.780656e-07,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,1
91748,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,10,0.403709,0.829567,1.847001e-08,...,0.483038,0.285413,-0.406400,0.570957,0.473768,-0.055774,0.396282,0.485782,-0.104788,0
91749,p11/Ciclo_5_5_Izquierda/Ciclo_5_5_Izquierda_z.mp4,p11,5,1,z,27,11,0.643907,0.492386,-1.789463e-07,...,0.480457,0.287692,-0.410172,0.570401,0.466859,-0.068966,0.398920,0.484171,-0.099403,1


In [ ]:
people = ['p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p07', 'p08', 'p09', 'p10', 'p11']

for p in people:
  p_df = model.full_df[model.full_df['person_id'] == p]
  p_df = p_df[p_df['active_hand'] == 1]
  p_df = p_df.drop(columns=[sup.tag_columns+['class_name'+'current_frame'+'active_hand']])

  X = p_df.drop(columns=['class_numeric'])
  y = p_df['class_numeric']
  

  model.me.eval()
  p_preds, p_labels = [], []
  p_logits = []
  with torch.no_grad():
    for xb, yb in model.test_loader:
        xb = xb.to(self.device)
        logits = self.me(xb)
        preds = logits.argmax(dim=1).cpu()
        all_preds.append(preds)
        all_labels.append(yb)
        all_logits.append(logits.cpu())

  self.y_true = torch.cat(all_labels)
  self.y_logits = torch.cat(all_logits)
  self.y_pred = torch.cat(all_preds)